In [1]:
import pandas as pd
import datetime as dt

from sqlalchemy import create_engine

In [2]:
file = "wowah_data.csv"

In [3]:
wow_df = pd.read_csv(file)

In [4]:
# check for and drop duplicate lines
print(len(wow_df))
print(len(wow_df.drop_duplicates()))

wow_df = wow_df.drop_duplicates()

10826734
10823177


In [5]:
wow_df.head()

,char,level,race,charclass,zone,guild,timestamp
0,59425,1,Orc,Rogue,Orgrimmar,165,01/01/08 00:02:04
1,65494,9,Orc,Hunter,Durotar,-1,01/01/08 00:02:04
2,65325,14,Orc,Warrior,Ghostlands,-1,01/01/08 00:02:04
3,65490,18,Orc,Hunter,Ghostlands,-1,01/01/08 00:02:04
4,2288,60,Orc,Hunter,Hellfire Peninsula,-1,01/01/08 00:02:09


In [6]:
# trimming whitespace from column headers
wow_df = wow_df.rename(columns=lambda x: x.strip())

In [7]:
# check for nan values
wow_df[wow_df.isna().any(axis=1)]

,char,level,race,charclass,zone,guild,timestamp


In [8]:
# check size of dataset
len(wow_df)

10823177

Because of the size of the dataset, I was running into issues performing basic df manipulations.
<br>
I decided to do some playing around with seeing the fastest way to manipulate the data.

In [9]:
# import tqdm to see progress_apply time
from tqdm._tqdm_notebook import tqdm_notebook

# initialize tqdm with pandas
tqdm_notebook.pandas()

C:\Users\tmkez\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [10]:
# use smaller df to test the progress_apply function
char_df = wow_df.loc[(wow_df['level'] > 70)]

char_df['timestamp'].progress_apply(pd.to_datetime)

9766188    2008-11-19 13:06:21
9766190    2008-11-19 13:06:21
9766194    2008-11-19 13:06:26
9766195    2008-11-19 13:06:41
9766226    2008-11-19 13:07:07
                   ...        
10826729   2008-12-31 23:50:18
10826730   2008-12-31 23:50:18
10826731   2008-12-31 23:50:18
10826732   2008-12-31 23:50:18
10826733   2008-12-31 23:50:18
Name: timestamp, Length: 716893, dtype: datetime64[ns]

In [11]:
%%timeit

# compare the progress_apply function to the to_datetime function

pd.to_datetime(char_df['timestamp'])

7.65 s ± 210 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We see the 'to_datetime' function is much faster than the apply version.

In [12]:
# convert the timestamp column from str to datetime
wow_df['timestamp'] = pd.to_datetime(wow_df['timestamp'])
wow_df.head()

,char,level,race,charclass,zone,guild,timestamp
0,59425,1,Orc,Rogue,Orgrimmar,165,2008-01-01 00:02:04
1,65494,9,Orc,Hunter,Durotar,-1,2008-01-01 00:02:04
2,65325,14,Orc,Warrior,Ghostlands,-1,2008-01-01 00:02:04
3,65490,18,Orc,Hunter,Ghostlands,-1,2008-01-01 00:02:04
4,2288,60,Orc,Hunter,Hellfire Peninsula,-1,2008-01-01 00:02:09


In [13]:
rds_connection_string = "postgres:password123@localhost:5432/wow_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

I have no idea how long this took, but it took a while!

In [14]:
wow_df.to_sql(name='wow_char_info', con=engine, if_exists='append', index=False, chunksize=25000, method=None)

In [16]:
%%timeit

# checked to see how long the to_datetime function would take on the entire dataset

# pd.to_datetime(wow_df['timestamp'])

3min 27s ± 35.7 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
